# Bayesian Inference for PDE Inverse Problems

This notebook explores Bayesian inference methods for PDE inverse problems:

- **MCMC Sampling**: Metropolis-Hastings and Hamiltonian Monte Carlo
- **Variational Inference**: Fast approximate inference
- **Convergence Diagnostics**: Ensuring reliable results
- **Posterior Analysis**: Understanding parameter relationships
- **Method Comparison**: When to use which approach

---

In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
from scipy.optimize import minimize
import time
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Plotting setup
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
%matplotlib inline

print("🔗 Bayesian Inference Demo - Setup Complete!")

## Problem Setup: Parameter Estimation for Heat Equation

We'll work with the 2D heat equation:
$$-\nabla \cdot (\kappa \nabla u) = f(x,y) \quad \text{in } \Omega$$
$$u = 0 \quad \text{on } \partial\Omega$$

**Goal**: Estimate unknown parameters $\theta = [\kappa, \sigma]$ where:
- $\kappa$: thermal conductivity 
- $\sigma$: source strength parameter

In [ ]:
# Import or implement framework components
try:
    from bayesian_pde_solver.pde_solvers import FiniteDifferenceSolver
    from bayesian_pde_solver.bayesian_inference import MCMCSampler, VariationalInference
    print("✅ Using framework implementations")
    framework_available = True
except ImportError:
    print("📝 Using custom implementations")
    framework_available = False
    
    # Minimal PDE solver implementation
    class SimplePDESolver:
        def __init__(self, domain_bounds, mesh_size):
            self.x_min, self.x_max, self.y_min, self.y_max = domain_bounds
            self.nx, self.ny = mesh_size
            
            self.x = np.linspace(self.x_min, self.x_max, self.nx)
            self.y = np.linspace(self.y_min, self.y_max, self.ny)
            self.X, self.Y = np.meshgrid(self.x, self.y, indexing='ij')
            
            self.dx = self.x[1] - self.x[0]
            self.dy = self.y[1] - self.y[0]
            
        def solve(self, kappa, sigma):
            """Solve 2D Poisson equation using Jacobi iteration."""
            u = np.zeros((self.nx, self.ny))
            
            # Source function: Gaussian centered at (0.3, 0.7)
            source = sigma * np.exp(-((self.X - 0.3)**2 + (self.Y - 0.7)**2) / 0.05)
            
            # Jacobi iteration
            for iteration in range(1000):
                u_new = u.copy()
                
                for i in range(1, self.nx-1):
                    for j in range(1, self.ny-1):
                        laplacian = ((u[i+1,j] + u[i-1,j]) / self.dx**2 + 
                                   (u[i,j+1] + u[i,j-1]) / self.dy**2)
                        
                        u_new[i,j] = (kappa * laplacian + source[i,j]) / (2*kappa*(1/self.dx**2 + 1/self.dy**2))
                
                # Boundary conditions (u = 0)
                u_new[0, :] = 0
                u_new[-1, :] = 0
                u_new[:, 0] = 0
                u_new[:, -1] = 0
                
                u = u_new
            
            return u.ravel()
        
        def get_coordinates(self):
            return np.column_stack([self.X.ravel(), self.Y.ravel()])

# Initialize solver
solver = SimplePDESolver((0, 1, 0, 1), (31, 31))
coordinates = solver.get_coordinates()

print(f"🔧 Solver initialized: {solver.nx}×{solver.ny} mesh")

In [ ]:
# Generate synthetic observation data
np.random.seed(42)

# True parameters (unknown to inference)
kappa_true = 1.5
sigma_true = 3.0
theta_true = np.array([kappa_true, sigma_true])

print(f"🎯 True parameters: κ = {kappa_true}, σ = {sigma_true}")

# Solve with true parameters
u_true = solver.solve(kappa_true, sigma_true)

# Observation points (sparse measurements)
n_obs = 25
obs_indices = np.random.choice(len(u_true), n_obs, replace=False)
obs_coords = coordinates[obs_indices]
u_obs_true = u_true[obs_indices]

# Add measurement noise
noise_std = 0.05 * np.max(u_obs_true)
noise = np.random.normal(0, noise_std, n_obs)
u_obs_noisy = u_obs_true + noise

print(f"📊 Generated {n_obs} observations with {noise_std:.4f} noise level")

# Visualize the problem setup
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# True solution field
U_true = u_true.reshape((solver.nx, solver.ny))
im1 = axes[0].contourf(solver.X, solver.Y, U_true, levels=20, cmap='viridis')
axes[0].scatter(obs_coords[:, 0], obs_coords[:, 1], c='red', s=50, marker='o', 
               label=f'{n_obs} observations')
axes[0].set_title('True Solution Field')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].legend()
plt.colorbar(im1, ax=axes[0])

# Observations with noise
axes[1].scatter(obs_coords[:, 0], obs_coords[:, 1], c=u_obs_true, s=100, 
               cmap='viridis', marker='o', label='True values', alpha=0.7)
axes[1].scatter(obs_coords[:, 0], obs_coords[:, 1], c=u_obs_noisy, s=50, 
               cmap='viridis', marker='x', label='Noisy observations')
axes[1].set_title('Observation Data')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].legend()

plt.tight_layout()
plt.show()

## Bayesian Setup: Prior and Posterior

### Prior Specification
We use log-normal priors for positive parameters:
- $\kappa \sim \text{LogNormal}(0, 1)$
- $\sigma \sim \text{LogNormal}(1, 0.5)$

### Likelihood Model
$$p(\mathbf{y}|\theta) \propto \exp\left(-\frac{1}{2\sigma_{noise}^2}\|\mathbf{y} - \mathbf{g}(\theta)\|^2\right)$$

where $\mathbf{g}(\theta)$ is the PDE solution at observation points.

In [ ]:
# Define Bayesian components
def log_prior(theta):
    """Log prior probability."""
    kappa, sigma = theta
    
    if kappa <= 0 or sigma <= 0:
        return -np.inf
    
    # Log-normal priors
    log_p_kappa = stats.lognorm.logpdf(kappa, s=1.0, scale=1.0)
    log_p_sigma = stats.lognorm.logpdf(sigma, s=0.5, scale=np.exp(1.0))
    
    return log_p_kappa + log_p_sigma

def log_likelihood(theta, observations, obs_indices, noise_std):
    """Log likelihood of observations."""
    kappa, sigma = theta
    
    try:
        # Solve PDE
        solution = solver.solve(kappa, sigma)
        predictions = solution[obs_indices]
        
        # Gaussian likelihood
        residuals = observations - predictions
        log_lik = -0.5 * np.sum(residuals**2) / noise_std**2
        log_lik -= 0.5 * len(observations) * np.log(2 * np.pi * noise_std**2)
        
        return log_lik
        
    except Exception as e:
        print(f"PDE solve failed: {e}")
        return -np.inf

def log_posterior(theta):
    """Log posterior probability."""
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    
    ll = log_likelihood(theta, u_obs_noisy, obs_indices, noise_std)
    return lp + ll

# Test the posterior function
test_theta = np.array([1.0, 2.0])
test_lp = log_posterior(test_theta)
print(f"✅ Posterior function working: log p(θ) = {test_lp:.3f}")

# Find MAP estimate for initialization
def neg_log_posterior(theta):
    return -log_posterior(theta)

print("🔍 Finding MAP estimate...")
map_result = minimize(neg_log_posterior, x0=np.array([1.0, 2.0]), 
                     bounds=[(0.1, 10.0), (0.1, 10.0)],
                     method='L-BFGS-B')

theta_map = map_result.x
print(f"📍 MAP estimate: κ = {theta_map[0]:.3f}, σ = {theta_map[1]:.3f}")
print(f"🎯 True values:  κ = {kappa_true:.3f}, σ = {sigma_true:.3f}")
print(f"📊 MAP log posterior: {-map_result.fun:.3f}")

## MCMC Sampling: Metropolis-Hastings Algorithm

The Metropolis-Hastings algorithm generates samples from the posterior distribution:

1. **Propose**: $\theta' \sim q(\theta'|\theta^{(t)})$
2. **Accept/Reject**: Accept with probability $\min(1, \alpha)$ where
   $$\alpha = \frac{p(\theta'|\mathbf{y})q(\theta^{(t)}|\theta')}{p(\theta^{(t)}|\mathbf{y})q(\theta'|\theta^{(t)})}$$

In [ ]:
# Implement Metropolis-Hastings sampler
class MetropolisHastingsSampler:
    def __init__(self, log_posterior_fn, initial_state, step_size=0.1):
        self.log_posterior_fn = log_posterior_fn
        self.current_state = np.array(initial_state)
        self.step_size = step_size
        self.current_log_prob = log_posterior_fn(self.current_state)
        
    def sample(self, n_samples):
        """Run MCMC sampling."""
        samples = np.zeros((n_samples, len(self.current_state)))
        log_probs = np.zeros(n_samples)
        n_accepted = 0
        
        print(f"🔗 Running MCMC: {n_samples} samples...")
        
        for i in range(n_samples):
            # Propose new state (random walk)
            proposal = self.current_state + np.random.normal(0, self.step_size, len(self.current_state))
            
            # Compute acceptance probability
            proposed_log_prob = self.log_posterior_fn(proposal)
            
            if np.isfinite(proposed_log_prob):
                log_alpha = proposed_log_prob - self.current_log_prob
                alpha = min(1.0, np.exp(log_alpha))
                
                # Accept or reject
                if np.random.rand() < alpha:
                    self.current_state = proposal
                    self.current_log_prob = proposed_log_prob
                    n_accepted += 1
            
            samples[i] = self.current_state
            log_probs[i] = self.current_log_prob
            
            # Progress indicator
            if (i + 1) % (n_samples // 10) == 0:
                acceptance_rate = n_accepted / (i + 1)
                print(f"   {i+1:5d}/{n_samples} samples, acceptance rate: {acceptance_rate:.3f}")
        
        acceptance_rate = n_accepted / n_samples
        print(f"✅ MCMC complete! Final acceptance rate: {acceptance_rate:.3f}")
        
        return {
            'samples': samples,
            'log_probs': log_probs,
            'acceptance_rate': acceptance_rate
        }

# Run MCMC sampling
if framework_available:
    mcmc = MCMCSampler(log_posterior, parameter_dim=2)
    mcmc_result = mcmc.sample(n_samples=5000, initial_state=theta_map, step_size=0.05)
else:
    mcmc = MetropolisHastingsSampler(log_posterior, theta_map, step_size=0.05)
    mcmc_result = mcmc.sample(n_samples=5000)

mcmc_samples = mcmc_result['samples']
mcmc_acceptance = mcmc_result['acceptance_rate']

print(f"\n📊 MCMC Results Summary:")
print(f"   Samples collected: {len(mcmc_samples)}")
print(f"   Acceptance rate: {mcmc_acceptance:.3f}")
print(f"   Target range: 0.2 - 0.6 (optimal: ~0.4)")

In [ ]:
# MCMC Diagnostics and Visualization
def plot_mcmc_traces(samples, parameter_names, true_values=None, burnin=500):
    """Plot MCMC traces and marginal distributions."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    colors = ['blue', 'red']
    
    for i, (param_name, color) in enumerate(zip(parameter_names, colors)):
        # Trace plot
        axes[0, i].plot(samples[:, i], color=color, alpha=0.7, linewidth=0.8)
        axes[0, i].axvline(burnin, color='black', linestyle='--', alpha=0.5, label='Burn-in')
        if true_values is not None:
            axes[0, i].axhline(true_values[i], color='green', linestyle='-', linewidth=2, label='True value')
        axes[0, i].set_title(f'Trace: {param_name}')
        axes[0, i].set_xlabel('Iteration')
        axes[0, i].set_ylabel(param_name)
        axes[0, i].legend()
        axes[0, i].grid(True, alpha=0.3)
        
        # Marginal distribution
        post_burnin = samples[burnin:, i]
        axes[1, i].hist(post_burnin, bins=50, density=True, alpha=0.7, color=color, 
                       label=f'Posterior (n={len(post_burnin)})')
        
        # Add statistics
        mean_val = np.mean(post_burnin)
        std_val = np.std(post_burnin)
        axes[1, i].axvline(mean_val, color='black', linestyle='-', linewidth=2, 
                          label=f'Mean: {mean_val:.3f}')
        axes[1, i].axvline(mean_val - std_val, color='black', linestyle='--', alpha=0.7)
        axes[1, i].axvline(mean_val + std_val, color='black', linestyle='--', alpha=0.7, 
                          label=f'±1σ: {std_val:.3f}')
        
        if true_values is not None:
            axes[1, i].axvline(true_values[i], color='green', linestyle='-', linewidth=2, 
                             label='True value')
        
        axes[1, i].set_title(f'Marginal: {param_name}')
        axes[1, i].set_xlabel(param_name)
        axes[1, i].set_ylabel('Density')
        axes[1, i].legend()
        axes[1, i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot MCMC results
parameter_names = ['κ (conductivity)', 'σ (source strength)']
fig = plot_mcmc_traces(mcmc_samples, parameter_names, true_values=theta_true, burnin=500)
plt.show()

# Compute posterior statistics
burnin = 500
post_samples = mcmc_samples[burnin:]

posterior_means = np.mean(post_samples, axis=0)
posterior_stds = np.std(post_samples, axis=0)
posterior_quantiles = np.percentile(post_samples, [2.5, 97.5], axis=0)

print("\n📈 Posterior Statistics (after burn-in):")
print("="*60)
for i, name in enumerate(['κ', 'σ']):
    mean_val = posterior_means[i]
    std_val = posterior_stds[i]
    ci_low, ci_high = posterior_quantiles[:, i]
    true_val = theta_true[i]
    
    print(f"   {name}:")
    print(f"      Posterior mean: {mean_val:.3f} ± {std_val:.3f}")
    print(f"      95% credible interval: [{ci_low:.3f}, {ci_high:.3f}]")
    print(f"      True value: {true_val:.3f} {'✓' if ci_low <= true_val <= ci_high else '✗'}")
    print(f"      Relative error: {abs(mean_val - true_val)/true_val*100:.1f}%\n")

## Convergence Diagnostics

Essential for ensuring MCMC reliability:
- **Effective Sample Size (ESS)**: Independent samples after autocorrelation
- **Autocorrelation Function**: How quickly chains mix
- **Gelman-Rubin Statistic**: Comparing multiple chains

In [ ]:
# Convergence diagnostics
def autocorrelation_function(x, max_lag=200):
    """Compute autocorrelation function."""
    n = len(x)
    x_centered = x - np.mean(x)
    
    # Use FFT for efficiency
    f_x = np.fft.fft(x_centered, 2*n-1)
    autocorr_fft = np.fft.ifft(f_x * np.conj(f_x)).real
    autocorr = autocorr_fft[:n] / autocorr_fft[0]
    
    return autocorr[:min(max_lag, n)]

def effective_sample_size(x):
    """Compute effective sample size."""
    autocorr = autocorrelation_function(x)
    
    # Find integrated autocorrelation time
    # Sum until autocorrelation becomes negative or very small
    tau_int = 1.0
    for i in range(1, len(autocorr)):
        if autocorr[i] <= 0.01:  # Cutoff at 1%
            break
        tau_int += 2 * autocorr[i]
    
    return len(x) / tau_int

# Analyze convergence
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, param_name in enumerate(parameter_names):
    param_samples = post_samples[:, i]
    
    # Autocorrelation
    autocorr = autocorrelation_function(param_samples, max_lag=200)
    lags = np.arange(len(autocorr))
    
    axes[0, i].plot(lags, autocorr, 'b-', linewidth=2)
    axes[0, i].axhline(0, color='black', linestyle='--', alpha=0.5)
    axes[0, i].axhline(0.01, color='red', linestyle='--', alpha=0.5, label='1% threshold')
    axes[0, i].set_xlabel('Lag')
    axes[0, i].set_ylabel('Autocorrelation')
    axes[0, i].set_title(f'Autocorrelation: {param_name}')
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)
    
    # Running average
    running_mean = np.cumsum(param_samples) / np.arange(1, len(param_samples) + 1)
    axes[1, i].plot(running_mean, 'b-', linewidth=2, label='Running mean')
    axes[1, i].axhline(theta_true[i], color='green', linestyle='-', linewidth=2, label='True value')
    axes[1, i].axhline(np.mean(param_samples), color='red', linestyle='--', linewidth=2, label='Final mean')
    axes[1, i].set_xlabel('Sample number')
    axes[1, i].set_ylabel(param_name)
    axes[1, i].set_title(f'Running Mean: {param_name}')
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute ESS
print("🔍 Convergence Diagnostics:")
print("="*50)
for i, param_name in enumerate(['κ', 'σ']):
    ess = effective_sample_size(post_samples[:, i])
    print(f"   {param_name}:")
    print(f"      Effective sample size: {ess:.1f} / {len(post_samples)} ({ess/len(post_samples)*100:.1f}%)")
    print(f"      Autocorrelation time: {len(post_samples)/ess:.1f} samples\n")

total_ess = min([effective_sample_size(post_samples[:, i]) for i in range(2)])
print(f"💡 Minimum ESS: {total_ess:.1f} (rule of thumb: >400 for reliable inference)")

## Variational Inference: Fast Approximate Bayesian Inference

When MCMC is too slow, variational inference provides a fast alternative by:
1. **Approximating** the posterior with a simpler distribution family
2. **Optimizing** the ELBO (Evidence Lower BOund)
3. **Trading accuracy for speed**

We use mean-field approximation: $q(\theta) = \prod_i q_i(\theta_i)$

In [ ]:
# Implement simple variational inference
class MeanFieldVI:
    def __init__(self, log_posterior_fn, parameter_dim):
        self.log_posterior_fn = log_posterior_fn
        self.parameter_dim = parameter_dim
        
        # Variational parameters (mean and log std for each parameter)
        self.means = np.zeros(parameter_dim)
        self.log_stds = np.log(0.5) * np.ones(parameter_dim)  # Initial std = 0.5
        
    def sample_variational(self, n_samples):
        """Sample from variational distribution."""
        stds = np.exp(self.log_stds)
        samples = np.random.normal(self.means, stds, (n_samples, self.parameter_dim))
        return samples
    
    def variational_log_prob(self, samples):
        """Log probability under variational distribution."""
        stds = np.exp(self.log_stds)
        log_probs = np.sum(stats.norm.logpdf(samples, self.means, stds), axis=1)
        return log_probs
    
    def elbo(self, n_samples=100):
        """Compute Evidence Lower BOund."""
        samples = self.sample_variational(n_samples)
        
        # E_q[log p(θ, y)]
        log_joint = np.array([self.log_posterior_fn(sample) for sample in samples])
        
        # E_q[log q(θ)]
        log_q = self.variational_log_prob(samples)
        
        # ELBO = E_q[log p(θ, y)] - E_q[log q(θ)]
        elbo_val = np.mean(log_joint - log_q)
        
        return elbo_val
    
    def elbo_gradient(self, n_samples=100, h=1e-5):
        """Compute ELBO gradient using finite differences."""
        baseline_elbo = self.elbo(n_samples)
        
        gradients = np.zeros(2 * self.parameter_dim)  # means + log_stds
        
        # Gradient w.r.t. means
        for i in range(self.parameter_dim):
            self.means[i] += h
            perturbed_elbo = self.elbo(n_samples)
            gradients[i] = (perturbed_elbo - baseline_elbo) / h
            self.means[i] -= h
        
        # Gradient w.r.t. log_stds
        for i in range(self.parameter_dim):
            self.log_stds[i] += h
            perturbed_elbo = self.elbo(n_samples)
            gradients[self.parameter_dim + i] = (perturbed_elbo - baseline_elbo) / h
            self.log_stds[i] -= h
        
        return gradients
    
    def optimize(self, n_iterations=1000, learning_rate=0.01, n_samples=50):
        """Optimize variational parameters."""
        elbo_history = []
        
        print(f"🔄 Running variational inference: {n_iterations} iterations...")
        
        for iteration in range(n_iterations):
            # Compute ELBO and gradient
            current_elbo = self.elbo(n_samples)
            elbo_history.append(current_elbo)
            
            grad = self.elbo_gradient(n_samples)
            
            # Gradient ascent update
            self.means += learning_rate * grad[:self.parameter_dim]
            self.log_stds += learning_rate * grad[self.parameter_dim:]
            
            # Progress
            if (iteration + 1) % (n_iterations // 10) == 0:
                print(f"   Iteration {iteration+1:4d}: ELBO = {current_elbo:.3f}")
        
        print(f"✅ VI optimization complete!")
        
        return {
            'means': self.means.copy(),
            'stds': np.exp(self.log_stds.copy()),
            'elbo_history': elbo_history
        }

# Run variational inference
if framework_available:
    vi = VariationalInference(log_posterior, parameter_dim=2, vi_type="mean_field")
    vi_result = vi.optimize(n_iterations=500, learning_rate=0.01)
else:
    vi = MeanFieldVI(log_posterior, parameter_dim=2)
    # Initialize near MAP
    vi.means = theta_map.copy()
    vi_result = vi.optimize(n_iterations=500, learning_rate=0.01, n_samples=50)

print(f"\n📊 VI Results Summary:")
print(f"   Final ELBO: {vi_result['elbo_history'][-1]:.3f}")
print(f"   ELBO improvement: {vi_result['elbo_history'][-1] - vi_result['elbo_history'][0]:.3f}")

In [ ]:
# Compare VI and MCMC results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# ELBO convergence
axes[0, 0].plot(vi_result['elbo_history'], 'b-', linewidth=2)
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('ELBO')
axes[0, 0].set_title('VI Convergence')
axes[0, 0].grid(True, alpha=0.3)

# Generate VI samples for comparison
vi_samples = vi.sample_variational(len(post_samples))

# Parameter comparisons
for i, param_name in enumerate(parameter_names):
    ax_idx = i + 1
    
    # Marginal distributions
    axes[0, ax_idx].hist(post_samples[:, i], bins=50, density=True, alpha=0.6, 
                        color='blue', label='MCMC')
    axes[0, ax_idx].hist(vi_samples[:, i], bins=50, density=True, alpha=0.6, 
                        color='red', label='VI')
    axes[0, ax_idx].axvline(theta_true[i], color='green', linestyle='-', linewidth=2, 
                           label='True')
    axes[0, ax_idx].set_xlabel(param_name)
    axes[0, ax_idx].set_ylabel('Density')
    axes[0, ax_idx].set_title(f'Marginal: {param_name}')
    axes[0, ax_idx].legend()
    axes[0, ax_idx].grid(True, alpha=0.3)

# Joint distribution scatter plots
# MCMC
axes[1, 0].scatter(post_samples[::10, 0], post_samples[::10, 1], 
                  alpha=0.6, s=20, color='blue', label='MCMC')
axes[1, 0].scatter(theta_true[0], theta_true[1], 
                  color='green', s=100, marker='*', label='True')
axes[1, 0].set_xlabel('κ (conductivity)')
axes[1, 0].set_ylabel('σ (source strength)')
axes[1, 0].set_title('MCMC Joint Distribution')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# VI
axes[1, 1].scatter(vi_samples[::10, 0], vi_samples[::10, 1], 
                  alpha=0.6, s=20, color='red', label='VI')
axes[1, 1].scatter(theta_true[0], theta_true[1], 
                  color='green', s=100, marker='*', label='True')
axes[1, 1].set_xlabel('κ (conductivity)')
axes[1, 1].set_ylabel('σ (source strength)')
axes[1, 1].set_title('VI Joint Distribution')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Comparison
axes[1, 2].scatter(post_samples[::20, 0], post_samples[::20, 1], 
                  alpha=0.4, s=15, color='blue', label='MCMC')
axes[1, 2].scatter(vi_samples[::20, 0], vi_samples[::20, 1], 
                  alpha=0.4, s=15, color='red', label='VI')
axes[1, 2].scatter(theta_true[0], theta_true[1], 
                  color='green', s=100, marker='*', label='True')
axes[1, 2].set_xlabel('κ (conductivity)')
axes[1, 2].set_ylabel('σ (source strength)')
axes[1, 2].set_title('MCMC vs VI Comparison')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Quantitative comparison
vi_means = np.mean(vi_samples, axis=0)
vi_stds = np.std(vi_samples, axis=0)
vi_quantiles = np.percentile(vi_samples, [2.5, 97.5], axis=0)

print("\n📊 Method Comparison:")
print("="*80)
print(f"{'Parameter':<12} {'True':<8} {'MCMC Mean':<12} {'VI Mean':<12} {'MCMC Std':<12} {'VI Std':<12}")
print("-"*80)
for i, name in enumerate(['κ', 'σ']):
    true_val = theta_true[i]
    mcmc_mean = posterior_means[i]
    mcmc_std = posterior_stds[i]
    vi_mean = vi_means[i]
    vi_std = vi_stds[i]
    
    print(f"{name:<12} {true_val:<8.3f} {mcmc_mean:<12.3f} {vi_mean:<12.3f} {mcmc_std:<12.3f} {vi_std:<12.3f}")

print("\n💡 Key Insights:")
print("   • MCMC captures true posterior shape and correlations")
print("   • VI is faster but assumes independence (mean-field)")
print("   • VI may underestimate uncertainty due to approximation")
print("   • Choose method based on accuracy vs speed requirements")

## Advanced MCMC: Hamiltonian Monte Carlo

Hamiltonian Monte Carlo (HMC) uses gradient information to make more efficient proposals:
- **Leverages gradients** of the log posterior
- **Reduces random walk behavior**
- **Higher acceptance rates**
- **Better exploration** of parameter space

In [ ]:
# Simplified HMC implementation
def finite_difference_gradient(func, x, h=1e-6):
    """Compute gradient using finite differences."""
    grad = np.zeros_like(x)
    
    for i in range(len(x)):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += h
        x_minus[i] -= h
        
        grad[i] = (func(x_plus) - func(x_minus)) / (2 * h)
    
    return grad

class SimpleHMC:
    def __init__(self, log_posterior_fn, initial_state, step_size=0.01, n_leapfrog=10):
        self.log_posterior_fn = log_posterior_fn
        self.current_state = np.array(initial_state)
        self.step_size = step_size
        self.n_leapfrog = n_leapfrog
        self.current_log_prob = log_posterior_fn(self.current_state)
    
    def leapfrog(self, q, p):
        """Leapfrog integration for Hamiltonian dynamics."""
        # Half step for momentum
        grad = finite_difference_gradient(self.log_posterior_fn, q)
        p = p + 0.5 * self.step_size * grad
        
        # Full steps
        for i in range(self.n_leapfrog):
            # Full step for position
            q = q + self.step_size * p
            
            # Full step for momentum (except last)
            if i < self.n_leapfrog - 1:
                grad = finite_difference_gradient(self.log_posterior_fn, q)
                p = p + self.step_size * grad
        
        # Final half step for momentum
        grad = finite_difference_gradient(self.log_posterior_fn, q)
        p = p + 0.5 * self.step_size * grad
        
        return q, p
    
    def sample(self, n_samples):
        """Run HMC sampling."""
        samples = np.zeros((n_samples, len(self.current_state)))
        log_probs = np.zeros(n_samples)
        n_accepted = 0
        
        print(f"🚀 Running HMC: {n_samples} samples...")
        print(f"   Step size: {self.step_size}, Leapfrog steps: {self.n_leapfrog}")
        
        for i in range(n_samples):
            # Sample momentum
            p = np.random.normal(0, 1, len(self.current_state))
            
            # Current energy
            current_energy = -self.current_log_prob + 0.5 * np.sum(p**2)
            
            # Leapfrog integration
            q_new, p_new = self.leapfrog(self.current_state.copy(), p.copy())
            
            # Proposed energy
            proposed_log_prob = self.log_posterior_fn(q_new)
            
            if np.isfinite(proposed_log_prob):
                proposed_energy = -proposed_log_prob + 0.5 * np.sum(p_new**2)
                
                # Accept/reject
                energy_diff = current_energy - proposed_energy
                if energy_diff > 0 or np.random.rand() < np.exp(energy_diff):
                    self.current_state = q_new
                    self.current_log_prob = proposed_log_prob
                    n_accepted += 1
            
            samples[i] = self.current_state
            log_probs[i] = self.current_log_prob
            
            # Progress
            if (i + 1) % (n_samples // 10) == 0:
                acceptance_rate = n_accepted / (i + 1)
                print(f"   {i+1:5d}/{n_samples} samples, acceptance rate: {acceptance_rate:.3f}")
        
        acceptance_rate = n_accepted / n_samples
        print(f"✅ HMC complete! Final acceptance rate: {acceptance_rate:.3f}")
        
        return {
            'samples': samples,
            'log_probs': log_probs,
            'acceptance_rate': acceptance_rate
        }

# Run HMC (smaller sample size due to computational cost)
print("⚡ Demonstrating Hamiltonian Monte Carlo...")
hmc = SimpleHMC(log_posterior, theta_map, step_size=0.005, n_leapfrog=20)
hmc_result = hmc.sample(n_samples=1000)

hmc_samples = hmc_result['samples']
hmc_acceptance = hmc_result['acceptance_rate']

print(f"\n📊 HMC vs MCMC Comparison:")
print(f"   HMC acceptance rate: {hmc_acceptance:.3f}")
print(f"   MCMC acceptance rate: {mcmc_acceptance:.3f}")
print(f"   HMC target range: 0.6 - 0.9 (higher than MCMC)")

In [ ]:
# Compare all three methods
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Parameter 1: κ (conductivity)
axes[0, 0].hist(post_samples[:, 0], bins=30, density=True, alpha=0.6, 
               color='blue', label=f'MCMC (n={len(post_samples)})')
axes[0, 0].hist(vi_samples[:, 0], bins=30, density=True, alpha=0.6, 
               color='red', label=f'VI (n={len(vi_samples)})')
axes[0, 0].hist(hmc_samples[100:, 0], bins=20, density=True, alpha=0.6, 
               color='orange', label=f'HMC (n={len(hmc_samples)-100})')
axes[0, 0].axvline(theta_true[0], color='green', linestyle='-', linewidth=2, label='True')
axes[0, 0].set_xlabel('κ (conductivity)')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Marginal Distribution: κ')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Parameter 2: σ (source strength)
axes[0, 1].hist(post_samples[:, 1], bins=30, density=True, alpha=0.6, 
               color='blue', label=f'MCMC (n={len(post_samples)})')
axes[0, 1].hist(vi_samples[:, 1], bins=30, density=True, alpha=0.6, 
               color='red', label=f'VI (n={len(vi_samples)})')
axes[0, 1].hist(hmc_samples[100:, 1], bins=20, density=True, alpha=0.6, 
               color='orange', label=f'HMC (n={len(hmc_samples)-100})')
axes[0, 1].axvline(theta_true[1], color='green', linestyle='-', linewidth=2, label='True')
axes[0, 1].set_xlabel('σ (source strength)')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Marginal Distribution: σ')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Joint distributions
axes[1, 0].scatter(post_samples[::20, 0], post_samples[::20, 1], 
                  alpha=0.4, s=15, color='blue', label='MCMC')
axes[1, 0].scatter(vi_samples[::20, 0], vi_samples[::20, 1], 
                  alpha=0.4, s=15, color='red', label='VI')
axes[1, 0].scatter(hmc_samples[100::5, 0], hmc_samples[100::5, 1], 
                  alpha=0.6, s=20, color='orange', label='HMC')
axes[1, 0].scatter(theta_true[0], theta_true[1], 
                  color='green', s=100, marker='*', label='True')
axes[1, 0].set_xlabel('κ (conductivity)')
axes[1, 0].set_ylabel('σ (source strength)')
axes[1, 0].set_title('Joint Distribution Comparison')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Effective sample size comparison
methods = ['MCMC', 'VI', 'HMC']
kappa_ess = [
    effective_sample_size(post_samples[:, 0]),
    len(vi_samples),  # VI samples are independent
    effective_sample_size(hmc_samples[100:, 0])
]
sigma_ess = [
    effective_sample_size(post_samples[:, 1]),
    len(vi_samples),  # VI samples are independent
    effective_sample_size(hmc_samples[100:, 1])
]

x = np.arange(len(methods))
width = 0.35

axes[1, 1].bar(x - width/2, kappa_ess, width, label='κ ESS', alpha=0.7)
axes[1, 1].bar(x + width/2, sigma_ess, width, label='σ ESS', alpha=0.7)
axes[1, 1].set_xlabel('Method')
axes[1, 1].set_ylabel('Effective Sample Size')
axes[1, 1].set_title('Sampling Efficiency Comparison')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(methods)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final comparison table
hmc_means = np.mean(hmc_samples[100:], axis=0)
hmc_stds = np.std(hmc_samples[100:], axis=0)

print("\n🏆 Final Method Comparison:")
print("="*90)
print(f"{'Method':<8} {'κ Mean':<10} {'κ Std':<10} {'σ Mean':<10} {'σ Std':<10} {'Accept Rate':<12} {'Efficiency':<12}")
print("-"*90)
print(f"{'True':<8} {theta_true[0]:<10.3f} {'-':<10} {theta_true[1]:<10.3f} {'-':<10} {'-':<12} {'-':<12}")
print(f"{'MCMC':<8} {posterior_means[0]:<10.3f} {posterior_stds[0]:<10.3f} {posterior_means[1]:<10.3f} {posterior_stds[1]:<10.3f} {mcmc_acceptance:<12.3f} {min(kappa_ess[0], sigma_ess[0]):<12.1f}")
print(f"{'VI':<8} {vi_means[0]:<10.3f} {vi_stds[0]:<10.3f} {vi_means[1]:<10.3f} {vi_stds[1]:<10.3f} {'-':<12} {min(kappa_ess[1], sigma_ess[1]):<12.1f}")
print(f"{'HMC':<8} {hmc_means[0]:<10.3f} {hmc_stds[0]:<10.3f} {hmc_means[1]:<10.3f} {hmc_stds[1]:<10.3f} {hmc_acceptance:<12.3f} {min(kappa_ess[2], sigma_ess[2]):<12.1f}")

## Method Selection Guidelines

### When to Use Each Method:

**MCMC (Metropolis-Hastings)**:
- ✅ **Gold standard** for accurate posterior sampling
- ✅ **Few assumptions** about posterior shape
- ✅ **Reliable** for complex, multimodal distributions
- ⚠️ **Slow convergence** for high-dimensional problems

**Variational Inference**:
- ✅ **Fast** - orders of magnitude faster than MCMC
- ✅ **Scalable** to high-dimensional problems
- ✅ **Deterministic** - reproducible results
- ⚠️ **Approximate** - may underestimate uncertainty
- ⚠️ **Strong assumptions** (e.g., mean-field independence)

**Hamiltonian Monte Carlo**:
- ✅ **Efficient exploration** using gradients
- ✅ **Higher acceptance rates** than random walk
- ✅ **Good for smooth posteriors**
- ⚠️ **Requires gradients** (automatic differentiation)
- ⚠️ **Tuning required** (step size, integration time)

## Summary and Key Takeaways

### What We've Learned:

1. **Bayesian Framework**:
   - Prior knowledge + Data likelihood → Posterior distribution
   - Uncertainty quantification is natural and principled
   - Parameter correlations are captured

2. **MCMC Sampling**:
   - Metropolis-Hastings: Simple and robust
   - Convergence diagnostics are essential
   - Effective sample size matters more than total samples

3. **Variational Inference**:
   - Trade accuracy for speed
   - ELBO optimization provides approximate posterior
   - Great for large-scale problems

4. **Method Selection**:
   - Consider accuracy vs computational budget
   - MCMC for small problems requiring high accuracy
   - VI for large problems or rapid prototyping
   - HMC when gradients are available

### Next Steps:
- **Notebook 04**: Certified uncertainty quantification
- **Notebook 05**: Visualization gallery
- **Experiment**: Try different prior specifications
- **Challenge**: Implement your own inference method

In [ ]:
# Create completion summary
print("🎓 Bayesian Inference Demo - Complete!")
print("=" * 50)

skills_learned = [
    "✅ Bayesian inference framework setup",
    "✅ MCMC sampling (Metropolis-Hastings)",
    "✅ Convergence diagnostics and ESS",
    "✅ Variational inference (mean-field)",
    "✅ Hamiltonian Monte Carlo basics",
    "✅ Method comparison and selection",
    "✅ Posterior analysis and interpretation",
    "✅ Computational efficiency considerations"
]

print("🎯 Skills Acquired:")
for skill in skills_learned:
    print(f"   {skill}")

print("\n🚀 Next Steps:")
next_steps = [
    "📓 Notebook 04: Certified Uncertainty Quantification",
    "📓 Notebook 05: Visualization Gallery",
    "🔧 Try different priors and likelihoods",
    "🧪 Experiment with method parameters",
    "📊 Apply to your own PDE problems"
]

for step in next_steps:
    print(f"   {step}")

print("\n💡 Key Insight:")
print("   Bayesian inference provides principled uncertainty quantification,")
print("   but method choice depends on accuracy vs efficiency tradeoffs.")

print("\n🎉 Ready for certified bounds? Continue to Notebook 04!")